In [23]:
from keras.models import load_model
import keras
import tensorflow as tf
import numpy as np


In [24]:
%run ../../layers/dimension_utils

In [25]:
def np2tf(x):
    if isinstance(x, np.ndarray):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        return x, False
    return x, True


def match_tensor(x1:tf.Tensor or np.ndarray, x2:tf.Tensor or np.ndarray):
    
    x1, f1 = np2tf(x1)
    x2, f2 = np2tf(x2)

    # no need to transpose if all var are tensor, we assume tensor are computed by gragh.
    if f1 and f2:
        return x1, x2
    
    # ensure tensor is set to x1, weights set to x2
    if f2:
        x1, x2 = x2, x1

    if x1.shape.ndims != x2.shape.ndims:
        while x2.shape.ndims < x1.shape.ndims:
            x2 = tf.expand_dims(x2, axis=0)
    
    new_shape = shape_NCD_to_NDC_format([i for i in range(len(x2.shape))])
    x2 = tf.transpose(x2, new_shape)
    return (x2, x1) if f2 else (x1, x2)

In [40]:
@keras.saving.register_keras_serializable()
class TFAdd(keras.layers.Layer):
    def __init__(self, tensor_grap, node_weights, node_inputs, *args, **kwargs):
        super().__init__()
        self.tensor_grap = tensor_grap
        self.node_weights = node_weights
        self.node_inputs = node_inputs
        # print('tensor grap: ', tensor_grap)
        print('type grap: ', type(tensor_grap))
        print('type weight: ', type(node_weights))
        print('type inputs: ', type(node_inputs))
        # print('node weight: ', node_weights)
        self.first_operand = tensor_grap[node_inputs[0]] if node_inputs[0] in tensor_grap else node_weights[node_inputs[0]]
        self.second_operand = tensor_grap[node_inputs[1]] if node_inputs[1] in tensor_grap else node_weights[node_inputs[1]]
        self.first_operand, self.second_operand = match_tensor(self.first_operand, self.second_operand)
        print("type first: ", type(self.first_operand))
        print('second: ', self.second_operand)
        print('asfas: ', tf.keras.backend.variable(self.second_operand['config']['keras_history']))

    def call(self, *args, **kwargs):
        print("first operand: ", self.first_operand)
        print("second operand: ", self.second_operand)
        return keras.Input(shape=self.first_operand['config']['shape'],
                                             dtype=self.first_operand['config']['dtype'])+ keras.Input(shape=self.second_operand['config']['shape'],
                                             dtype=self.second_operand['config']['dtype'])
    def get_config(self):
        config = super().get_config()
        config.update({
            "tensor_grap":self.tensor_grap,
            'node_weights':self.node_weights,
            'node_inputs':self.node_inputs
        })
        return config
    
class TFLog(keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super().__init__()
    def call(self, inputs, *args, **kwargs):
        return keras.ops.log(inputs)

from keras import backend as K
K.clear_session()
keras.saving.get_custom_objects().clear()
custom_objects = {"TFAdd": TFAdd, "TFLog":TFLog}
with keras.saving.custom_object_scope(custom_objects):
    model = load_model("add.keras")
    model.summary()



type grap:  <class 'dict'>
type weight:  <class 'dict'>
type inputs:  <class 'list'>
type first:  <class 'keras.src.utils.tracking.TrackedDict'>
second:  {'class_name': '__keras_tensor__', 'config': {'shape': [1, 3, 1], 'dtype': 'float32', 'keras_history': ['input_layer_1', 0, 0]}}


TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional_1', 'trainable': False, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [1, 3, 1], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'layers.mathematics_layers', 'class_name': 'TFLog', 'config': {'name': 'tf_log', 'index': 0, 'trainable': False, 'dtype': 'float32'}, 'registered_name': 'TFLog', 'build_config': {'input_shape': [1, 3, 1]}, 'name': 'tf_log', 'inbound_nodes': [{'args': [{'class_name': '__keras_tensor__', 'config': {'shape': [1, 3, 1], 'dtype': 'float32', 'keras_h

In [36]:
import torch
input1 = torch.tensor([10, 40, 50], dtype = torch.float32).reshape(1,-1,1)
input2 = torch.tensor([13, 4, 7], dtype = torch.float32).reshape(1,-1,1)
model.predict((input1, input2))

first operand:  {'class_name': '__keras_tensor__', 'config': {'shape': [1, 3, 1], 'dtype': 'float32', 'keras_history': ['tf_log', 0, 0]}}
second operand:  {'class_name': '__keras_tensor__', 'config': {'shape': [1, 3, 1], 'dtype': 'float32', 'keras_history': ['input_layer_1', 0, 0]}}


TypeError: To be compatible with tf.function, Python functions must return zero or more Tensors or ExtensionTypes or None values; in compilation of <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x17fc29120>, found return value of type KerasTensor, which is not a Tensor or ExtensionType.